Treinamento com o modelo Mask-RCNN 

In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config


ROOT_DIR = "./maskrcnn_model/"
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
#Configuração para treinamento no conjunto de dados 
class COCOConfig(Config):
    NAME = "CocoDataset_config"

    IMAGES_PER_GPU = 40

    NUM_CLASSES = 1 + 80  # Background + restante das classes

    #Número de etapas de treinamento por época
    STEPS_PER_EPOCH = 10


In [ ]:
#método para obter os nomes das classes
def class_name():
    path = open("./annotations/instances_train2017.json", "r")
    anns = json.load(path)
    name_cat = []

    for name in anns['categories']['name']:
        name_cat.append(name)

    return name_cat

#método para obter os ids das classes
def class_id():
    path = open("./annotations/instances_train2017.json", "r")
    anns = json.load(path)
    id_cat = []

    for id in anns['categories']['id']:
        id_cat.append(id)
        
    return id_cat

In [ ]:
#Nessa classe se inicia a personalização do conjunto de dados
class COCODataset(utils.Dataset):
    def load_custom(self, dataset_dir, subset):
        source_name = "object"

        #criando as variaveis que contém o id das categorias e os nomes
        id_cat=list, name_cat= list
        id_cat = class_id()
        name_cat = class_name()

        #adicionando as categorias na classe
        for x in range(0, 79):
            self.add_class(source_name, id_cat[x], name_cat[x])

        #adicionando os dados pertencentes às imgs de treino
        if(subset == "train"):
            annot = json.load(open('./annotations/instances_train2017.json'))       
            images_dir = dataset_dir + 'images/train/all_images'
            #obtendo o filename da imagem
            for filename in os.listdir(images_dir):
                image_id = filename[:-4]

        #adicionando os dados pertencentes às imgs de validação
        else:
            annot = json.load(open('./annotations/instances_val2017.json'))
            images_dir = dataset_dir + 'images/val/all_images'
            #obtendo o filename da imagem
            for filename in os.listdir(images_dir):
                image_id = filename[:-4]

        #carregando informações da imagem
        img_path = images_dir + filename
        ann_img = annot['annotations'][image_id]['image_id']
        image_width = image_id['width']
        image_height = image_id['height']
               
        
        self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=img_path,
                    width=image_width,
                    height=image_height,
                    annotations=ann_img
                )

    #Carregando máscaras de instância para a imagem fornecida
    def load_mask(self, image_id):
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)
 
        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids
 

In [ ]:
#iniciando o treinamento
train_dataset = COCODataset()
train_dataset.load_dataset(dataset_dir='./BD/', subset='train')
train_dataset.prepare()

#iniciando a validação
val_dataset = COCODataset()
val_dataset.load_dataset(dataset_dir='./BD/', subset='val')
val_dataset.prepare()


config = COCOConfig()

model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

start_train = time.time()
model.train(train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            learning_rate=config.LEARNING_RATE, 
            epochs=100, 
            layers='all')


In [ ]:
#precisão média (mAP) - TREINAMENTO
total_gt = np.array([]) 
total_pred = np.array([]) 
mAP_ = [] #mAP list


for image_id in train_dataset.image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask =modellib.load_image_gt(train_dataset, config, image_id)
    info = train_dataset.image_info[image_id]

    #percorrendo o modelo
    results = model.detect([image], verbose=1)
    r = results[0]
    
    #compute gt_tot and pred_tot
    gt, pred = utils.gt_pred_lists(gt_class_id, gt_bbox, r['class_ids'], r['rois'])
    total_gt = np.append(total_gt, gt)
    total_pred = np.append(total_pred, pred)
    
    #precision_, recall_, AP_ 
    AP_, precision_, recall_, overlap_ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])


    print("O atual comprimento do vetor de previsão é: ", len(total_pred))
    
    mAP_.append(AP_)
    print("Precisão média desta imagem: ",AP_)
    print("A precisão média real para todas as imagens de treino é:", sum(mAP_)/len(mAP_))

import pandas as pd
total_gt=total_gt.astype(int)
total_pred=total_pred.astype(int)
#salvando os vetores de gt e pred
save_dir = "output"
gt_pred_tot_json = {"Verdade total" : total_gt, "caixa prevista" : total_pred}
df = pd.DataFrame(gt_pred_tot_json)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
df.to_json(os.path.join(save_dir,"gt_pred_test.json"))

In [ ]:
#precisão média (mAP) - VALIDAÇÃO
total_gt = np.array([]) 
total_pred = np.array([]) 
mAP_ = [] #mAP list


for image_id in val_dataset.image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask =modellib.load_image_gt(val_dataset, config, image_id)
    info = val_dataset.image_info[image_id]

    #percorrendo o modelo
    results = model.detect([image], verbose=1)
    r = results[0]
    
    #compute gt_tot and pred_tot
    gt, pred = utils.gt_pred_lists(gt_class_id, gt_bbox, r['class_ids'], r['rois'])
    total_gt = np.append(total_gt, gt)
    total_pred = np.append(total_pred, pred)
    
    #precision_, recall_, AP_ 
    AP_, precision_, recall_, overlap_ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])


    print("O atual comprimento do vetor de previsão é: ", len(total_pred))
    
    mAP_.append(AP_)
    print("Precisão média desta imagem: ",AP_)
    print("A precisão média real para todas as imagens de validação é:", sum(mAP_)/len(mAP_))

import pandas as pd
total_gt=total_gt.astype(int)
total_pred=total_pred.astype(int)
#salvando os vetores de gt e pred
save_dir = "output"
gt_pred_tot_json = {"Verdade total" : total_gt, "caixa prevista" : total_pred}
df = pd.DataFrame(gt_pred_tot_json)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
df.to_json(os.path.join(save_dir,"gt_pred_val.json"))